In [5]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from omegaconf import OmegaConf

from superlimo.lib import get_n, get_destimation_domain, warp, clip_and_pad_images, keypoints2position, get_pm_grids, pattern_matching, apply_pm_corrections
from superlimo.superlimo import SuperLIMo
from superlimo.matcher import Matcher
%matplotlib inline

In [6]:
conf = OmegaConf.load('example_config.yml')

In [7]:
f0 = '../images/S1A_EW_GRDM_1SDH_20150121T132623_20150121T132723_004270_005316_D11B.SAFE'
f1 = '../images/S1A_EW_GRDM_1SDH_20150123T063756_20150123T063856_004295_0053AD_73C7.SAFE'

n0 = get_n(f0)
n1 = get_n(f1)
dst_dom = get_destimation_domain(conf.proj4, conf.extent, conf.sar_resolution)
image_time_delta = (n1.time_coverage_start - n0.time_coverage_start).total_seconds()

In [8]:
d = {}
d['hh0'] = warp(n0, n0[1], dst_dom)
d['hh1'] = warp(n1, n1[1], dst_dom)
d['hv0'] = warp(n0, n0[2], dst_dom)
d['hv1'] = warp(n1, n1[2], dst_dom)
d = clip_and_pad_images(d, conf.min_sar_signal, conf.plim)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
axs[0].imshow(d['hh0'], cmap='gray')
axs[1].imshow(d['hh1'], cmap='gray')
plt.show()

In [10]:
img0 = torch.tensor(np.stack([d['hh0'], d['hv0']], 0))[None].float()
img1 = torch.tensor(np.stack([d['hh1'], d['hv1']], 0))[None].float()
img0[img0.isnan()] = 0
img1[img1.isnan()] = 0


In [11]:
superlimo = SuperLIMo(conf)
with torch.no_grad():
    pre0 = superlimo(img0)
    pre1 = superlimo(img1)


In [12]:
pos0 = keypoints2position(pre0['keypoints'], dst_dom)
pos1 = keypoints2position(pre1['keypoints'], dst_dom)

In [13]:
matcher = Matcher(plot=False, time_delta=image_time_delta, **conf)
idx0, idx1, model = matcher.match(pos0, pos1, pre0['descriptors'].numpy().T, pre1['descriptors'].numpy().T)

In [ ]:
plt.quiver(pos0[idx0, 0], pos0[idx0, 1], pos1[idx1, 0] - pos0[idx0, 0], pos1[idx1, 1] - pos0[idx0, 1], angles='xy', scale_units='xy', scale=1)
plt.colorbar()

In [15]:
c0pm, r0pm, x0pm, y0pm, c1pmfg, r1pmfg, gpi_pm = get_pm_grids(model, dst_dom, conf.pm_step, conf.pm_template_size, conf.pm_border, conf.proj4)

In [16]:
corrections = pattern_matching(d, c0pm, r0pm, c1pmfg, r1pmfg, gpi_pm, conf.pm_template_size, conf.pm_border, conf.pm_pol)

In [17]:
x1pm, y1pm, c1pm, r1pm, mccpm = apply_pm_corrections(corrections, c1pmfg, r1pmfg, gpi_pm, dst_dom)

In [ ]:
gpi = mccpm > 0.4
plt.imshow(d['hh0'], extent=[x0pm.min(), x0pm.max(), y0pm.min(), y0pm.max()], cmap='gray')
plt.quiver(x0pm[gpi], y0pm[gpi], x1pm[gpi] - x0pm[gpi], y1pm[gpi] - y0pm[gpi], mccpm[gpi], angles='xy', scale_units='xy', scale=3, cmap='jet')